In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Yellow Feature Engineering")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 20:37:04 WARN Utils: Your hostname, MIchaels-HP resolves to a loopback address: 127.0.1.1; using 172.20.216.58 instead (on interface eth0)
23/08/19 20:37:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 20:37:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 20:37:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/19 20:37:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/08/19 20:37:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.sql import functions as F

In [3]:
yellow_sdf = spark.read.parquet('../data/landing/yellow_tlc_clean')

Make the features we want to be used for analysis

In [4]:
#Add a cloumn that is just the trip date

yellow_sdf = yellow_sdf.withColumn('trip_date', F.to_date('tpep_pickup_datetime'))

In [5]:
#Find what day of week it is, this range is [1,7] (1 = Sunday, ..., 7 = Saturday) and if it is the weekend or not

yellow_sdf = yellow_sdf.withColumn('day_of_week', F.dayofweek('tpep_pickup_datetime'))\
                       .withColumn('is_weekend', F.dayofweek('tpep_pickup_datetime').isin([1,7]).cast('int'))


In [6]:
#Calculate the time each ride took

yellow_sdf = yellow_sdf.withColumn('FROM', F.to_timestamp('tpep_pickup_datetime'))\
                    .withColumn("UNTILL", F.to_timestamp('tpep_dropoff_datetime'))\
                    .withColumn('trip_time_min', F.round((F.col('UNTILL').cast("long") - F.col("FROM").cast("long"))/60, 3))\
                    .drop("FROM","UNTILL")

In [7]:
#Calculate the fair/minute for each ride 

yellow_sdf = yellow_sdf.withColumn('fare/minute', F.round(F.col("fare_amount")/F.col("trip_time_min"), 4))

In [8]:
#Check if the trip happend between 12 am and 6 am, 6 am and 10 am or 5pm and 8pm 
yellow_sdf = yellow_sdf.withColumn('timestamp', F.to_timestamp('tpep_pickup_datetime')) \
                       .withColumn('hour', F.hour('timestamp'))\
                       .withColumn('morning_0_to_6', F.col('hour').isin([x for x in range(0,6)]).cast('int'))\
                       .withColumn('morning_6_to_10', F.col('hour').isin([x for x in range(6,10)]).cast('int'))\
                       .withColumn('evening_17_to_20', F.col('hour').isin([x for x in range(17,20)]).cast('int'))\
                       .drop('timestamp')

Save the data to be used for analysis

In [9]:
yellow_sdf.write.mode('overwrite').parquet('../data/curated/yellow_tlc_final')

23/08/19 20:40:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/19 20:40:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/19 20:40:47 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
